In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


!pip install tensorflow-datasets

In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [6]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for s, l in train_data:
    train_sentences.append(s.numpy().decode('utf8'))
    train_labels.append(l.numpy())
    
for s, l in test_data:
    test_sentences.append(s.numpy().decode('utf8'))
    test_labels.append(l.numpy())

train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

In [8]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [10]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(train_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 10
model.fit(padded, train_labels_final, epochs=num_epochs, validation_data=(testing_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 3ms/step - loss: 0.4914 - accuracy: 0.7478 - val_loss: 0.3396 - val_accuracy: 0.8530
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2488 - accuracy: 0.9014 - val_loss: 0.3588 - val_accuracy: 0.8444
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1088 - accuracy: 0.9701 - val_loss: 0.4553 - val_accuracy: 0.8230
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0298 - accuracy: 0.9958 - val_loss: 0.5366 - val_accuracy: 0.8246
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0084 - accuracy: 0.9993 - val_loss: 0.6001 - val_accuracy: 0.8260
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0025 - accuracy: 0.9999 - val_loss: 0.6584 - val_accuracy: 0.8270
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.7049 - val_accuracy: 0.8274
Epoch 

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [15]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [16]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
